In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from itertools import product
import collections
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import time
import pandas as pd
import json
from IPython.display import clear_output
import import_dataset as load
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
import copy


# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, AdaptiveAvgPool2d
from torch.optim import Adam, SGD

In [2]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
#Importation du data
train_path = "../dataset/train" 
test_path = "../dataset/test" 

train_dataset_X, train_dataset_Y = load.loadDataset(train_path)
test_dataset_X, test_dataset_Y = load.loadDataset(test_path)

print(train_dataset_X.shape)
print(train_dataset_Y.shape)

debut chargement image
debut chargement image
(318, 3, 224, 224)
(318,)


In [4]:
print(test_dataset_X.shape)
print(test_dataset_Y.shape)

(318, 3, 224, 224)
(318,)


In [5]:
# Encoding Y labels

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

train_dataset_Y = encoder.fit_transform(train_dataset_Y)
test_dataset_Y = encoder.fit_transform(test_dataset_Y)

print(np.unique(test_dataset_Y))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]


In [18]:
#Create custom dataLoader

class DogsDataset(Dataset):
    """ Dogs dataset."""
    # Initialize your data, download, etc.
    def __init__(self, X, Y):
        self.len = Y.shape[0]
        self.x_data = torch.from_numpy(X).float()
        self.y_data = torch.from_numpy(Y).float()
        print(self.x_data.shape)
        print(self.y_data.shape)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


dataset_train = DogsDataset(train_dataset_X, train_dataset_Y)
dataset_test = DogsDataset(test_dataset_X, test_dataset_Y)

train_loader = DataLoader(dataset=dataset_train,
                          batch_size=100,
                          shuffle=True,
                          num_workers=0)

test_loader = DataLoader(dataset=dataset_test,
                          shuffle=True,
                          batch_size=1,
                          num_workers=0)

torch.Size([318, 3, 224, 224])
torch.Size([318])
torch.Size([318, 3, 224, 224])
torch.Size([318])


In [7]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            Conv2d(3, 10, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(10),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(10, 20, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(20),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(62720, 120) # A  changer soon 
        )
  
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [10]:
class Inception(Module):

    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()

        self.branch1 = Sequential(
            Conv2d(in_channels, ch1x1, bias=False, kernel_size=1),
            BatchNorm2d(ch1x1, eps=0.001),
            ReLU(inplace=True)
        )
        
        self.branch2 = Sequential(
            Conv2d(in_channels, ch3x3red, bias=False, kernel_size=1),
            BatchNorm2d(ch3x3red, eps=0.001),
            ReLU(inplace=True),
            Conv2d(ch3x3red, ch3x3, bias=False, kernel_size=3, padding=1),
            BatchNorm2d(ch3x3, eps=0.001),
            ReLU(inplace=True)
        )

        self.branch3 = Sequential(
            Conv2d(in_channels, ch5x5red, bias=False, kernel_size=1),
            BatchNorm2d(ch5x5red, eps=0.001),
            ReLU(inplace=True),
            Conv2d(ch5x5red, ch5x5, bias=False, kernel_size=3, padding=1),
            # Here, kernel_size=3 instead of kernel_size=5 is a known bug.
            # Please see https://github.com/pytorch/vision/issues/906 for details.
            BatchNorm2d(ch5x5, eps=0.001),
            ReLU(inplace=True)
        )

        self.branch4 = Sequential(
            MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            Conv2d(in_channels, pool_proj, bias=False, kernel_size=1),
            BatchNorm2d(pool_proj, eps=0.001),
            ReLU(inplace=True)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)


In [11]:
class GoogLeNet(Module):
    
    def __init__(self):
        super(GoogLeNet, self).__init__()
        
        self.conv1 = Conv2d(3, 64, bias=False, kernel_size=7, stride=2, padding=3)
        self.bn1 = BatchNorm2d(64, eps=0.001)
        self.maxpool1 = MaxPool2d(3, stride=2, ceil_mode=True)
        
        self.conv2 = Conv2d(64, 64, bias=False, kernel_size=1)
        self.bn2 = BatchNorm2d(64, eps=0.001)
        
        self.conv3 = Conv2d(64, 192, bias=False, kernel_size=3, padding=1)
        self.bn3 = BatchNorm2d(192, eps=0.001)
        self.maxpool2 = MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = MaxPool2d(2, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        self.relu = ReLU(inplace=True)
        self.avgpool = AdaptiveAvgPool2d((1, 1))
        self.dropout = Dropout(0.2)
        self.fc = Linear(1024, 120)
    
    
    def forward(self, x):       # N x 3 x 224 x 224
        
        x = self.conv1(x)       # N x 64 x 112 x 112
        x = self.bn1(x)         # N x 64 x 112 x 112
        x = self.relu(x)        # N x 64 x 112 x 112
        x = self.maxpool1(x)    # N x 64 x 56 x 56
        
        x = self.conv2(x)       # N x 64 x 56 x 56
        x = self.bn3(x)         # N x 64 x 56 x 56
        x = self.relu(x)        # N x 64 x 56 x 56

        x = self.conv3(x)       # N x 192 x 56 x 56
        x = self.bn3(x)         # N x 192 x 56 x 56
        x = self.relu(x)        # N x 192 x 56 x 56
        x = self.maxpool2(x)    # N x 192 x 28 x 28

        x = self.inception3a(x) # N x 256 x 28 x 28
        x = self.inception3b(x) # N x 480 x 28 x 28
        x = self.maxpool3(x)    # N x 480 x 14 x 14
        
        x = self.inception4a(x) # N x 512 x 14 x 14
        x = self.inception4b(x) # N x 512 x 14 x 14
        x = self.inception4c(x) # N x 512 x 14 x 14
        x = self.inception4d(x) # N x 528 x 14 x 14

        x = self.inception4e(x) # N x 832 x 14 x 14
        x = self.maxpool4(x)    # N x 832 x 7 x 7
        x = self.inception5a(x) # N x 832 x 7 x 7
        x = self.inception5b(x) # N x 1024 x 7 x 7

        x = self.avgpool(x)     # N x 1024 x 1 x 1
        x = torch.flatten(x, 1) # N x 1024
        x = self.dropout(x)
        x = self.fc(x)          # N x 120

        return x

In [12]:
#model = Net()
model = GoogLeNet()

optimizer = Adam(model.parameters(), lr=0.07)

criterion = CrossEntropyLoss()
    
print(model)

GoogLeNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (branch2): Sequential(
      (0): 

)


In [ ]:
def train(epoch):
    model.train()
    tr_loss = 0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data
        
        optimizer.zero_grad()
        output_train = model(inputs)

        print("output_train")
        print(output_train)
        print("labels")
        print(labels)
        loss_train = criterion(output_train, labels.long())
        train_losses.append(loss_train)

        loss_train.backward()
        optimizer.step()
        tr_loss += loss_train.item()
        # i1 = i1 + 100
        # i2 = i2 + 100
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_train)
    return tr_loss

In [ ]:
# defining the number of epochs
n_epochs = 100
train_losses = []

for epoch in range(n_epochs):
    train(epoch)
torch.save(model.state_dict(), 'model.pth')

In [ ]:
# Testing and predictions : 

test_dataset_X  = torch.from_numpy(test_dataset_X).float()
test_dataset_Y = torch.from_numpy(test_dataset_Y).float()

print(test_dataset_X.shape)
print(test_dataset_Y.shape)

In [ ]:
# loading the model
model = Net()
model.load_state_dict(torch.load('./model.pth'))
# generating predictions for test set
# output = model(test_dataset_X)

# softmax = torch.nn.Softmax(output)
# prob = list(softmax.numpy())
# predictions = np.argmax(prob, axis=1)

# # accuracy on training set
# accuracy_score(train_y, predictions)

correct = 0
total = 0
with torch.no_grad():
        outputs = model(test_dataset_X)
        # predicted = torch.nn.Softmax(outputs)
        _, predicted = torch.max(outputs, 1)
        total += test_dataset_Y.size(0)
        correct += (predicted == test_dataset_Y).sum().item()

In [ ]:
test_dataset_X

In [ ]:
predicted

In [ ]:
outputs